In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from selenium import webdriver
import main
import visuals
import seaborn as sns
import chromedriver_autoinstaller
from datetime import datetime
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch
from matplotlib.colors import to_rgba
import matplotlib as mpl
from collections import defaultdict

In [ ]:
chromedriver_autoinstaller.install()

## Pull Data

In [ ]:
if __name__ == "__main__":
    driver = webdriver.Chrome()
    
# whoscored match centre url of the required match (Example: Barcelona vs Sevilla)
url = "https://www.whoscored.com/Matches/1640876/Live/England-Premier-League-2022-2023-Arsenal-Manchester-United"
match_data = main.getMatchData(driver, url, close_window=True)

# Match dataframe containing info about the match
matches_df = main.createMatchesDF(match_data)

# Events dataframe      
events_df = main.createEventsDF(match_data)

# match Id
matchId = match_data['matchId']

# Information about respective teams as dictionary
home_data = matches_df['home'][matchId]
away_data = matches_df['away'][matchId]
score = matches_df['score'][matchId]
score = ' ' + score.replace(':', '-') + ' '

home_name = home_data['name']
away_name = away_data['name']
date = match_data['startDate'].split('T')[0]
date_ = datetime.strptime(date, '%Y-%m-%d').date()
date_str = date_.strftime('%d/%m/%Y')

## Filter data to get passes
- Whoscored doesn't have pass recipient - so need to add automatically
- Just shift index by one
- Also have instances where pass is successful but recipient is to another team (half end/free kicks?) - so make sure team id or recipient and player is the same

In [ ]:
events_df["passRecipient"] = events_df['playerName'].shift(-1)
events_df['teamRecipient'] = events_df['teamId'].shift(-1)
column_to_move = events_df.pop("passRecipient")
events_df.insert(26, "passRecipient", column_to_move)
column_to_move = events_df.pop("teamRecipient")
events_df.insert(27, "teamRecipient", column_to_move)

In [ ]:
events_df2 = events_df[(events_df['type'] == 'Pass')]

In [ ]:
events_df = events_df[(events_df['type'] == 'Pass') & (events_df['outcomeType'] == 'Successful')
                      & (events_df['teamId'] == events_df['teamRecipient']) ]

## Passes between 2 players

In [ ]:
passes_df = events_df[['eventId','playerName', 'passRecipient']]
passes_df = passes_df.dropna()

In [ ]:
passes_df['pair'] = passes_df['playerName'] + ' ' + passes_df['passRecipient']
passes_df

In [ ]:
passes_count = passes_df.groupby(['pair']).count().reset_index()
passes_count = passes_count[['pair', 'eventId']]
passes_count.columns = ['pair', 'passes']
passes_count

## Average position for each player

In [ ]:
avg_loc_df = events_df[['teamId', 'playerName', 'x', 'y']]
avg_loc_df = avg_loc_df.groupby(['teamId', 'playerName']).agg({'x':np.mean, 'y':[np.mean, 'count']}).reset_index()
avg_loc_df.columns = ['teamId', 'playerName', 'x','y', 'count']
avg_loc_df

## Merge Data  

In [ ]:
passes_merge = passes_df.merge(passes_count, on='pair')
passes_merge = passes_merge[['playerName', 'passRecipient', 'passes']]
passes_merge = passes_merge.drop_duplicates().reset_index(drop=True)
passes_merge

In [ ]:
avg_loc_df = avg_loc_df[['playerName', 'x', 'y', 'count']]
pass_map = passes_merge.merge(avg_loc_df, on='playerName')
pass_map.rename({'x':'x_start', 'y':'y_start'}, axis='columns', inplace=True)
pass_map = pass_map.merge(avg_loc_df, left_on = 'passRecipient', right_on='playerName', suffixes = ['', '_end'])
pass_map.rename({'x':'x_end', 'y':'y_end'}, axis='columns', inplace=True)
pass_map = pass_map.drop(['playerName_end', 'count_end'], axis=1)
pass_map = pass_map[pass_map['playerName'] != pass_map['passRecipient']]
pass_map

## Keep T1 Players

In [ ]:
id1 = home_data['teamId']
id2 = away_data['teamId']

In [ ]:
team1_players = events_df[events_df.teamId == id1].groupby('playerName').agg({'minute': [min, max]}).reset_index()
team1_players = pd.concat([team1_players['playerName'], team1_players['minute']], axis=1)
team1_players['mins_played'] = team1_players['max'] - team1_players['min']
team1_players = team1_players.sort_values('mins_played', ascending=False)
team1_players_names = team1_players.playerName
passes_t1 = pass_map[(pass_map['playerName'].isin(team1_players_names)) & 
                     (pass_map['passRecipient'].isin(team1_players_names))]
team1_players_names_top11 = team1_players.playerName[:11].tolist()
team1_players_names_top11

## Top 3 players with most completed passes

In [ ]:
comp_passes = passes_t1.groupby('playerName')[['playerName','passes']].sum().reset_index().sort_values(by='passes',
                                                                                        ascending=False)
comp_passes = comp_passes[comp_passes['playerName'].isin(team1_players_names_top11)][:3]
team1_players_names_top11
comp_passes_list = []
for p, passes in zip(comp_passes['playerName'], comp_passes['passes']):
    comp_passes_list.append([p, passes])
comp_passes_list

## Top 3 Combinations

In [ ]:
combos = passes_t1.sort_values(by='passes', ascending=False)
combos = combos[(combos['playerName'].isin(team1_players_names_top11)) & (combos['passRecipient'].isin(team1_players_names_top11))]
combos = combos[:3][['playerName', 'passRecipient', 'passes']]
combos_list = []
for p1, p2, cnt in zip(combos['playerName'], combos['passRecipient'], combos['passes']):
    combos_list.append([p1,p2, cnt])
combos_list

## Prog. Pass Field + Top 3 Prog. Passes

In [ ]:
prog_passes = events_df[['playerName', 'passRecipient', 'type', 'x', 'y', 'endX', 'endY']]
prog_passes.rename({'x':'x_start', 'y':'y_start', 'endX': 'x_end', 'endY':'y_end'}, axis='columns', inplace=True)
prog_passes = prog_passes[(prog_passes['playerName'].isin(team1_players_names))]
prog_passes = prog_passes.reset_index(drop=True)

prog_passes['Progressive'] = ''
for i in range(len(prog_passes)):
    x_0 = 120*(prog_passes['x_start'][i])/100
    y_0 = 80*(prog_passes['y_start'][i])/100
    x_1 = 120*(prog_passes['x_end'][i])/100
    y_1 = 80*(prog_passes['y_end'][i])/100
    
    beg_dist = np.sqrt((120 - x_0)**2 + (40 - y_0)**2)
    end_dist = np.sqrt((120 - x_1)**2 + (40 - y_1)**2)

    if end_dist/beg_dist <= 0.75:
        prog_passes['Progressive'][i] = 'True'
    else:
        prog_passes['Progressive'][i] = 'False'

        
prog_passes2 = prog_passes[prog_passes['Progressive'] == 'True'].groupby('playerName')['type'].count().sort_values(ascending=False).reset_index()
prog_passes2 = prog_passes2[prog_passes2['playerName'].isin(team1_players_names_top11)][:3]
prog_passes_list = []
for p, cnt in zip(prog_passes2['playerName'], prog_passes2['type']):
    prog_passes_list.append([p,cnt])

prog_received = pd.DataFrame(prog_passes[prog_passes['Progressive'] == 'True'].groupby('passRecipient')['type'].count().sort_values(ascending=False)).reset_index()
prog_received = prog_received[prog_received['passRecipient'].isin(team1_players_names_top11)][:3]
prog_received_list = []
for p, cnt in zip(prog_received['passRecipient'], prog_received['type']):
    prog_received_list.append([p,cnt])


## Top 3 Pass Completion

In [ ]:
events_df3 = events_df2[events_df2['playerName'].isin(team1_players_names_top11)][['playerName','outcomeType']]
pass_comp_df = pd.DataFrame(events_df3.groupby(['playerName', 'outcomeType']).size()).unstack(fill_value=0).stack().reset_index()
pass_comp_df.columns = ['playerName', 'outcome', 'count']
pass_comp = []
for i in range(0, len(pass_comp_df), 2):
    rate = pass_comp_df['count'][i] / (pass_comp_df['count'][i]+pass_comp_df['count'][i+1])
    pass_comp.append([pass_comp_df['playerName'][i], rate])

pass_comp.sort(key=lambda x:x[1], reverse=True)
pass_comp = pass_comp[0:3]
for i, val in enumerate(pass_comp):
    val[1] = round(float(val[1])*100)
pass_comp
    

## Top 3 xThreat via Pass

In [ ]:
# events_df_xT = events_df[['playerName', 'x', 'y', 'endX', 'endY']].reset_index(drop=True)

# xT_list = []
# xT = pd.read_csv("xT_grid.csv", header=None)
# xT = np.array(xT)
# xT_rows, xT_cols = xT.shape

# for name in team1_players_names_top11:
#     test = events_df_xT[events_df_xT['playerName'] == name]
#     test['x1_bin'] = pd.cut(test['x'], bins=xT_cols, labels=False)
#     test['x2_bin'] = pd.cut(test['endX'], bins=xT_cols, labels=False)
#     test['y1_bin'] = pd.cut(test['y'], bins=xT_rows, labels=False)
#     test['y2_bin'] = pd.cut(test['endY'], bins=xT_rows, labels=False)
#     test['xT_start'] = test[['x1_bin', 'y1_bin']].apply(lambda x:xT[x[1]][x[0]], axis=1)
#     test['xT_end'] = test[['x2_bin', 'y2_bin']].apply(lambda x:xT[x[1]][x[0]], axis=1)
#     test['xT_diff'] = test['xT_end'] - test['xT_start']
#     xT_added = test['xT_diff'].sum()
    
#     xT_list.append([name, xT_added])
    
# xT_list.sort(key=lambda x:x[1], reverse=True)
# xT_list2 = xT_list.copy()
# xT_list = xT_list[:3]
# for i, val in enumerate(xT_list):
#     val[1] = round(float(val[1]),2)
# xT_list2

In [ ]:
events_df_xT = events_df[['playerName', 'x', 'y', 'endX', 'endY']].reset_index(drop=True)

xT_list = []
path = "https://karun.in/blog/data/open_xt_12x8_v1.json"
xT = pd.read_json(path)
xT = np.array(xT)
xT_rows, xT_cols = xT.shape

for name in team1_players_names_top11:
    test = events_df_xT[events_df_xT['playerName'] == name]
    test['x1_bin'] = pd.cut(test['x'], bins=xT_cols, labels=False)
    test['x2_bin'] = pd.cut(test['endX'], bins=xT_cols, labels=False)
    test['y1_bin'] = pd.cut(test['y'], bins=xT_rows, labels=False)
    test['y2_bin'] = pd.cut(test['endY'], bins=xT_rows, labels=False)
    test['xT_start'] = test[['x1_bin', 'y1_bin']].apply(lambda x:xT[x[1]][x[0]], axis=1)
    test['xT_end'] = test[['x2_bin', 'y2_bin']].apply(lambda x:xT[x[1]][x[0]], axis=1)
    test['xT_diff'] = test['xT_end'] - test['xT_start']
    xT_added = test['xT_diff'].sum()
    
    xT_list.append([name, xT_added])
    
xT_list.sort(key=lambda x:x[1], reverse=True)
xT_list2 = xT_list.copy()
xT_list = xT_list[:3]
for i, val in enumerate(xT_list):
    val[1] = round(float(val[1]),2)
xT_list

In [ ]:
xT_scaled = []
minXT = abs(min(xT_list2, key=lambda x:x[1])[1])
for player, xT in xT_list2:
    xT_updated = xT + minXT + 0.05
    xT_scaled.append([player, xT_updated])
df_xT = pd.DataFrame(xT_scaled, columns=['playerName', 'xThreat'])
df_xT

## Keep 11 players with most mins

In [ ]:
# Get line width and just count instances of 4/more passes
team1_players_names = team1_players.playerName[:11].tolist()
passes_t1 = pass_map[(pass_map['playerName'].isin(team1_players_names)) & 
                     (pass_map['passRecipient'].isin(team1_players_names))]
passes_t1['width'] = passes_t1['passes']/passes_t1['passes'].max() * 20
passes_t1

In [ ]:
# Set marker size and keep over 4 passes
# passes_t1 = passes_t1[passes_t1['passes']>=2]
passes_t1['passes'][passes_t1['passes']<3] = 0
passes_t1['marker_size'] = (passes_t1['count']
                                         / passes_t1['count'].max() * 12500)
passes_t1

In [ ]:
passes_t1 = passes_t1.merge(df_xT, on='playerName')
passes_t1

## Get Shirt Numbers and merge to passes df 

In [ ]:
shirt_nos = matches_df['home'][matchId]['formations'][0]['jerseyNumbers']
playerIds = matches_df['home'][matchId]['formations'][0]['playerIds']
shirt_nos = pd.DataFrame({'no':shirt_nos, 'playerId':playerIds})
playerIds = events_df[['playerId', 'playerName']]
passes_t1 = passes_t1.merge(playerIds, on='playerName')
shirt_nos['playerId'] = shirt_nos['playerId'].astype(str)
passes_t1 = passes_t1.merge(shirt_nos, on='playerId')
passes_t1 = passes_t1.drop_duplicates().reset_index(drop=True)
passes_t1

## Repeat above for away team 

In [ ]:
team2_players = events_df[events_df.teamId == id2].groupby('playerName').agg({'minute': [min, max]}).reset_index()
team2_players = pd.concat([team2_players['playerName'], team2_players['minute']], axis=1)
team2_players['mins_played'] = team2_players['max'] - team2_players['min']
team2_players = team2_players.sort_values('mins_played', ascending=False)
team2_players_names = team2_players.playerName
passes_t2 = pass_map[(pass_map['playerName'].isin(team2_players_names)) & 
                     (pass_map['passRecipient'].isin(team2_players_names))]
team2_players_names_top11 = team2_players.playerName[:11].tolist()

comp_passes2 = passes_t2.groupby('playerName')[['playerName','passes']].sum().reset_index().sort_values(by='passes',
                                                                                        ascending=False)
comp_passes2 = comp_passes2[comp_passes2['playerName'].isin(team2_players_names_top11)][:3]
comp_passes_list2 = []
for p, passes in zip(comp_passes2['playerName'], comp_passes2['passes']):
    comp_passes_list2.append([p, passes])
comp_passes_list2

combos2 = passes_t2.sort_values(by='passes', ascending=False)
combos2 = combos2[(combos2['playerName'].isin(team2_players_names_top11)) & (combos2['passRecipient'].isin(team2_players_names_top11))]
combos2 = combos2[:3][['playerName', 'passRecipient', 'passes']]
combos_list2 = []
for p1, p2, cnt in zip(combos2['playerName'], combos2['passRecipient'], combos2['passes']):
    combos_list2.append([p1,p2, cnt])
combos_list2

prog_passes_away = events_df[['playerName', 'passRecipient', 'type', 'x', 'y', 'endX', 'endY']]
prog_passes_away.rename({'x':'x_start', 'y':'y_start', 'endX': 'x_end', 'endY':'y_end'}, axis='columns', inplace=True)
prog_passes_away = prog_passes_away[(prog_passes_away['playerName'].isin(team2_players_names_top11))]
prog_passes_away = prog_passes_away.reset_index(drop=True)

prog_passes_away['Progressive'] = ''
for i in range(len(prog_passes_away)):
    x_0 = 120*(prog_passes_away['x_start'][i])/100
    y_0 = 80*(prog_passes_away['y_start'][i])/100
    x_1 = 120*(prog_passes_away['x_end'][i])/100
    y_1 = 80*(prog_passes_away['y_end'][i])/100
    
    beg_dist = np.sqrt((120 - x_0)**2 + (40 - y_0)**2)
    end_dist = np.sqrt((120 - x_1)**2 + (40 - y_1)**2)

    if end_dist/beg_dist <= 0.75:
        prog_passes_away['Progressive'][i] = 'True'
    else:
        prog_passes_away['Progressive'][i] = 'False'

        
prog_passes_away2 = prog_passes_away[prog_passes_away['Progressive'] == 'True'].groupby('playerName')['type'].count().sort_values(ascending=False).reset_index()
prog_passes_away2 = prog_passes_away2[prog_passes_away2['playerName'].isin(team2_players_names_top11)][:3]
prog_passes_list2 = []
for p, cnt in zip(prog_passes_away2['playerName'], prog_passes_away2['type']):
    prog_passes_list2.append([p,cnt])

prog_received2 = pd.DataFrame(prog_passes_away[prog_passes_away['Progressive'] == 'True'].groupby('passRecipient')['type'].count().sort_values(ascending=False)).reset_index()
prog_received2 = prog_received2[prog_received2['passRecipient'].isin(team2_players_names_top11)][:3]
prog_received_list2 = []
for p, cnt in zip(prog_received2['passRecipient'], prog_received2['type']):
    prog_received_list2.append([p,cnt])

events_df4 = events_df2[events_df2['playerName'].isin(team2_players_names_top11)][['playerName','outcomeType']]
pass_comp_df2 = pd.DataFrame(events_df4.groupby(['playerName', 'outcomeType']).size()).unstack(fill_value=0).stack().reset_index()
pass_comp_df2.columns = ['playerName', 'outcome', 'count']
pass_comp_df2                         
pass_comp2 = []
for i in range(0, len(pass_comp_df2), 2):
    rate = pass_comp_df2['count'][i] / (pass_comp_df2['count'][i]+pass_comp_df2['count'][i+1])
    pass_comp2.append([pass_comp_df2['playerName'][i], rate])

pass_comp2.sort(key=lambda x:x[1], reverse=True)
pass_comp2 = pass_comp2[0:3]
for i, val in enumerate(pass_comp2):
    val[1] = round(float(val[1])*100)

events_df_xT = events_df[['playerName', 'x', 'y', 'endX', 'endY']].reset_index(drop=True)

xT_list3 = []
path = "https://karun.in/blog/data/open_xt_12x8_v1.json"
xT = pd.read_json(path)
xT = np.array(xT)
xT_rows, xT_cols = xT.shape

for name in team2_players_names_top11:
    test = events_df_xT[events_df_xT['playerName'] == name]
    test['x1_bin'] = pd.cut(test['x'], bins=xT_cols, labels=False)
    test['x2_bin'] = pd.cut(test['endX'], bins=xT_cols, labels=False)
    test['y1_bin'] = pd.cut(test['y'], bins=xT_rows, labels=False)
    test['y2_bin'] = pd.cut(test['endY'], bins=xT_rows, labels=False)
    test['xT_start'] = test[['x1_bin', 'y1_bin']].apply(lambda x:xT[x[1]][x[0]], axis=1)
    test['xT_end'] = test[['x2_bin', 'y2_bin']].apply(lambda x:xT[x[1]][x[0]], axis=1)
    test['xT_diff'] = test['xT_end'] - test['xT_start']
    xT_added = test['xT_diff'].sum()
    
    xT_list3.append([name, xT_added])
    
xT_list3.sort(key=lambda x:x[1], reverse=True)
xT_list4 = xT_list3.copy()
xT_list3 = xT_list3[:3]
for i, val in enumerate(xT_list3):
    val[1] = round(float(val[1]),2)

xT_scaled2 = []
minXT2 = abs(min(xT_list4, key=lambda x:x[1])[1])
for player, xT in xT_list4:
    xT_updated = xT + minXT + 0.05
    xT_scaled2.append([player, xT_updated])
df_xT2 = pd.DataFrame(xT_scaled2, columns=['playerName', 'xThreat'])

# Get line width and just count instances of 4/more passes
team2_players_names = team2_players.playerName[:11].tolist()
passes_t2 = pass_map[(pass_map['playerName'].isin(team2_players_names)) & 
                     (pass_map['passRecipient'].isin(team2_players_names))]
passes_t2['width'] = passes_t2['passes']/passes_t2['passes'].max() * 20

# Set marker size and keep over 4 passes
# passes_t1 = passes_t1[passes_t1['passes']>=2]
passes_t2['passes'][passes_t2['passes']<3] = 0
passes_t2['marker_size'] = (passes_t2['count']
                                         / passes_t2['count'].max() * 12500)
passes_t2 = passes_t2.merge(df_xT2, on='playerName')

shirt_nos2 = matches_df['away'][matchId]['formations'][0]['jerseyNumbers']
playerIds2 = matches_df['away'][matchId]['formations'][0]['playerIds']
shirt_nos2 = pd.DataFrame({'no':shirt_nos2, 'playerId':playerIds2})
playerIds2 = events_df[['playerId', 'playerName']]
passes_t2 = passes_t2.merge(playerIds2, on='playerName')
shirt_nos2['playerId'] = shirt_nos2['playerId'].astype(str)
passes_t2 = passes_t2.merge(shirt_nos2, on='playerId')
passes_t2 = passes_t2.drop_duplicates().reset_index(drop=True)
passes_t2

## Get Shot Data
- For the moment we're just pulling from Understat - plan is to build own xG model and get data from there, but just pulling from elsewhere at the minute
- All you need to change is the gameweek and rest should be automatic

In [ ]:
from understatapi import UnderstatClient

In [ ]:
understat = UnderstatClient()
match_id_h = understat.team(team=str(home_name)).get_match_data(season="2022")[20]['id']
shot_data_h = understat.match(match=str(match_id_h)).get_shot_data()
home_shots = pd.DataFrame([], columns=['X', 'Y', 'xG'])
away_shots = pd.DataFrame([], columns=['X', 'Y', 'xG'])
pitch4 = Pitch(pitch_type='opta')

In [ ]:
for i in range(len(shot_data_h['h'])):
    x = float(shot_data_h['h'][i]['X'])*100
    y = float(shot_data_h['h'][i]['Y'])*100
    xg = float(shot_data_h['h'][i]['xG'])
    row = {'X':x, 'Y':y, 'xG':xg}
    home_shots = home_shots.append(row, ignore_index=True)
home_shots['X'] = pitch4.dim.right - home_shots['X']

for i in range(len(shot_data_h['a'])):
    x = float(shot_data_h['a'][i]['X'])*100
    y = float(shot_data_h['a'][i]['Y'])*100
    xg = float(shot_data_h['a'][i]['xG'])
    row = {'X':x, 'Y':y, 'xG':xg}
    away_shots = away_shots.append(row, ignore_index=True)

## Metrics

In [ ]:
metrics = []

# Shots
home_shots_count = len(shot_data_h['h'])
away_shots_count = len(shot_data_h['a'])
metrics.append(['Shots', home_shots_count, away_shots_count])

# Big Chances (>=0.3xG)
big_chance_h = 0
for i in range(len(shot_data_h['h'])):
    if float(shot_data_h['h'][i]['xG']) >= 0.3:
        big_chance_h += 1
        
big_chance_a = 0
for i in range(len(shot_data_h['a'])):
    if float(shot_data_h['a'][i]['xG']) >= 0.3:
        big_chance_a += 1

metrics.append(['Big Chances', big_chance_h, big_chance_a])

# Open Play and Set Piece xG
opxg_h = 0
spxg_h = 0

for i in range(len(shot_data_h['h'])):
    if shot_data_h['h'][i]['situation'] == 'SetPiece' or shot_data_h['h'][i]['situation'] == 'FromCorner':
        spxg_h += float(shot_data_h['h'][i]['xG'])
    elif shot_data_h['h'][i]['situation'] == 'OpenPlay':
        opxg_h += float(shot_data_h['h'][i]['xG'])
        
opxg_h = round(opxg_h, 2)
spxg_h = round(spxg_h, 2)


opxg_a = 0
spxg_a = 0

for i in range(len(shot_data_h['a'])):
    if shot_data_h['a'][i]['situation'] == 'SetPiece' or shot_data_h['a'][i]['situation'] == 'FromCorner':
        spxg_a += float(shot_data_h['a'][i]['xG'])
    elif shot_data_h['a'][i]['situation'] == 'OpenPlay':
        opxg_a += float(shot_data_h['a'][i]['xG'])
        
opxg_a = round(opxg_a, 2)
spxg_a = round(spxg_a, 2)

metrics.append(['Open Play xG', opxg_h, opxg_a])
metrics.append(['Set Piece xG', spxg_h, spxg_a])

# Possession
home_poss = sum(matches_df['home'][matchId]['stats']['possession'].values())
away_poss = sum(matches_df['away'][matchId]['stats']['possession'].values())

home_poss_val = home_poss/(home_poss+away_poss)
home_poss_val = round(home_poss_val, 2)*100
away_poss_val = 100-home_poss_val
metrics.append(['Posession', int(home_poss_val), int(away_poss_val)])

# Passes
passes_h = sum(matches_df['home'][matchId]['stats']['passesTotal'].values())
passes_a = sum(matches_df['away'][matchId]['stats']['passesTotal'].values())
metrics.append(['Passes', int(passes_h), int(passes_a)])

# Corners 
corners_h = sum(matches_df['home'][matchId]['stats']['cornersTotal'].values())
corners_a = sum(matches_df['away'][matchId]['stats']['cornersTotal'].values())
metrics.append(['Corners', int(corners_h), int(corners_a)])

# Yellow and Red Cards 
yc_h = 0
rc_h = 0
for i in range(len(matches_df['home'][matchId]['incidentEvents'])):
    if matches_df['home'][matchId]['incidentEvents'][i]['type']['displayName'] == 'Card':
        if matches_df['home'][matchId]['incidentEvents'][i]['cardType']['displayName'] == 'Yellow':
            yc_h += 1
        else:
            rc_h += 1
            
# Fouls 
fouls_h = sum(matches_df['home'][matchId]['stats']['foulsCommited'].values())
fouls_a = sum(matches_df['away'][matchId]['stats']['foulsCommited'].values())
metrics.append(['Fouls', int(fouls_h), int(fouls_a)])


## Plot Dashboard 

In [ ]:
FIGWIDTH, FIGHEIGHT = 80, 35
FIGSIZE = (FIGWIDTH, FIGHEIGHT)
FIG_ASPECT = FIGWIDTH / FIGHEIGHT
fig = plt.figure(figsize=FIGSIZE)
# mpl.rcParams['figure.dpi'] = 800

MIN_TRANSPARENCY = 0.
color = np.array(to_rgba('darkcyan'))
color = np.tile(color, (len(passes_t1), 1))
c_transparency = passes_t1.passes / (passes_t1.passes.max())
c_transparency = (c_transparency * (1 - MIN_TRANSPARENCY)) + MIN_TRANSPARENCY
color[:, 3] = c_transparency

MIN_TRANSPARENCY2 = 0.05
color2 = np.array(to_rgba('darkcyan'))
color2 = np.tile(color2, (len(passes_t1), 1))
c_transparency2 = (passes_t1.xThreat + abs(passes_t1.xThreat.min()))/ ((passes_t1.xThreat.max() + abs(passes_t1.xThreat.min())))
c_transparency2 = (c_transparency2 + MIN_TRANSPARENCY2)/3
color2[:, 3] = c_transparency2

MIN_TRANSPARENCY3 = 0.0
color3 = np.array(to_rgba('mediumorchid'))
color3 = np.tile(color3, (len(passes_t2), 1))
c_transparency3 = passes_t2.passes / (passes_t2.passes.max())
c_transparency3 = (c_transparency3 * (1 - MIN_TRANSPARENCY3)) + MIN_TRANSPARENCY3
color3[:, 3] = c_transparency3

MIN_TRANSPARENCY4 = 0.05
color4 = np.array(to_rgba('mediumorchid'))
color4 = np.tile(color4, (len(passes_t2), 1))
c_transparency4 = (passes_t2.xThreat + abs(passes_t2.xThreat.min()))/ ((passes_t2.xThreat.max() + abs(passes_t2.xThreat.min())))
c_transparency4 = (c_transparency4 + MIN_TRANSPARENCY4)/3
color4[:, 3] = c_transparency4

PAD = 30
pitch_spec = {'pad_bottom': PAD, 'pad_top': PAD, 
              'pitch_color':'#2B2B2B', 'line_color':'#c7d5cc'}
 
pitch_width, pitch_length = 80, 60
pitch_width2, pitch_length2 = 30, 50


pitch1 = VerticalPitch(pitch_type='opta', pitch_width=pitch_width, pitch_length=pitch_length, 
                       pad_left = 40, pad_right = 0, **pitch_spec)
pitch2 = Pitch(pitch_type='opta', pitch_width=pitch_width2, pitch_length=pitch_length2, **pitch_spec)
pitch3 = VerticalPitch(pitch_type='opta', pitch_width=pitch_width, pitch_length=pitch_length, **pitch_spec,
                       pad_left=0, pad_right=40)

fig.set_facecolor("#2B2B2B")

TITLE_HEIGHT = 0.1 # title axes are 10% of the figure height

#  width of pitch axes as percent of the figure width
TOP_WIDTH = 0.3
TOP_WIDTH2 = 0.246
BOTTOM_WIDTH = 0.18

# calculate the horizontal space between axes (and figure sides) in percent of the figure width
TOP_SPACE = (1 - (TOP_WIDTH * 3)) / 4
BOTTOM_SPACE = (1 - (BOTTOM_WIDTH * 3)) / 4

# calculate the height of the pitch axes in percent of the figure height
height1 = (TOP_WIDTH / pitch1.ax_aspect * FIG_ASPECT)
height2 = (TOP_WIDTH2 / pitch2.ax_aspect * FIG_ASPECT)
height3 = TOP_WIDTH / pitch3.ax_aspect * FIG_ASPECT


# calculate pitch offsets from center / title locations
vertical_axes_space = (1 - (height1 + TITLE_HEIGHT + TITLE_HEIGHT)) / 5
bottom_offset = ((1 - height2) / 2) - vertical_axes_space
title1_bottom = 1 - vertical_axes_space - TITLE_HEIGHT
title2_bottom = 1 - (vertical_axes_space * 3) - (TITLE_HEIGHT * 2) - height1
top_offset = (1 - title1_bottom - title2_bottom - TITLE_HEIGHT) / 2



# top left
LEFT1 = TOP_SPACE
bottom1 = (1 - height1) / 2 - top_offset
ax1 = fig.add_axes((LEFT1, bottom1, TOP_WIDTH, height1))
pitch1.draw(ax=ax1)

# bottom middle
left2 = (TOP_SPACE * 2) + TOP_WIDTH + (TOP_WIDTH-TOP_WIDTH2)/2
bottom2 = bottom1 + 0.0425
ax2 = fig.add_axes((left2, bottom2, TOP_WIDTH2, height2))
pitch2.draw(ax=ax2)

# top right
left3 = (TOP_SPACE * 3) + (TOP_WIDTH)*2  
bottom3 = (1 - height3) / 2 - top_offset
ax3 = fig.add_axes((left3, bottom3, TOP_WIDTH, height3))
pitch3.draw(ax=ax3)


ax_title2 = fig.add_axes((0, title1_bottom-0.05, 1, TITLE_HEIGHT-0.05))
ax_title2.axis('off')
ax_title2.text(left2 + TOP_WIDTH2/2 ,0.95,
               f'{home_name} {score} { away_name}'.upper(),
               color='white', size=80, va='center', ha='center', fontname = 'Sans Serif')
ax_title2.text(left2 + TOP_WIDTH2/2 ,0.15,
               f'{date_str}  |  CREATED BY ...',
               color='white', size=60, va='center', ha='center', fontname = 'Sans Serif')
# ax_title2.set_xlim(0, 1)
# ax_title2.set_ylim(0, 1)
       
pitch1.annotate(text='  Passes Completed   '.upper(), xytext=(98.5, 120), xy=(60, 40), ha='center', 
                     va='center', color='white',
                     bbox=dict(facecolor='darkcyan', edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=35, ax = ax1)

y = 95
i = 0
for p1, cnt in comp_passes_list:
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    pitch1.annotate(text=p1.upper(), 
                          xytext=(y-2*i, 137.5), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax = ax1)
    pitch1.annotate(text=cnt, 
                          xytext=(y-2*i, 102), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax = ax1)
    
    i += 1
    
pitch1.annotate(text=' Pass Completion %  '.upper(), xytext=(83.5, 120), xy=(60, 40), ha='center', va='center', color='white',
                     bbox=dict(facecolor='darkcyan', edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=35, ax = ax1)
y = 80
i = 0
for p1, cnt in pass_comp:
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    pitch1.annotate(text=p1.upper(), 
                          xytext=(y-2*i, 137.5), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax = ax1)
    pitch1.annotate(text=str(cnt)+'%', 
                          xytext=(y-2*i, 102), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax = ax1)
    
    i += 1



pitch1.annotate(text='   Prog Passes Made '.upper(), xytext=(68.5, 120), xy=(60, 40), ha='center', va='center', color='white',
                     bbox=dict(facecolor='darkcyan', edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=35, ax = ax1)
y = 65
i = 0
for p1, cnt in prog_passes_list:
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    pitch1.annotate(text=p1.upper(), 
                          xytext=(y-2*i, 137.5), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax = ax1)
    pitch1.annotate(text=cnt, 
                          xytext=(y-2*i, 103), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax = ax1)
    
    i += 1

pitch1.annotate(text='    Prog Passes Rec   '.upper(), xytext=(53.5, 120), xy=(60, 40), ha='center', va='center', color='white',
                     bbox=dict(facecolor='darkcyan', edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=36, ax = ax1)
y = 50
i = 0
for p1, cnt in prog_received_list:
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    pitch1.annotate(text=p1.upper(), 
                          xytext=(y-2*i, 137.5), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax=ax1)
    pitch1.annotate(text=cnt, 
                          xytext=(y-2*i, 102), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax=ax1)
    
    i += 1


pitch1.annotate(text=' Pass Combinations  '.upper(), xytext=(38.5, 120), xy=(60, 40), ha='center', va='center', color='white',
                     bbox=dict(facecolor='darkcyan', edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=36, ax=ax1)
y = 35
i = 0
for p1, p2, cnt in combos_list:
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
        
    p2 = p2.replace('-', ' ')
    p2 = p2.split(' ')
    if len(p2) > 2:
        name = "".join([x[0].upper() for x in p2])
        p2 = name
    elif len(p2) == 1:
        p2 = p2[0]
    else:
        p2 = p2[1]

    pitch1.annotate(text=p1.upper() + ' --> ' + p2.upper(), 
                          xytext=(y-2*i, 137.5), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax=ax1)
    pitch1.annotate(text=cnt, 
                          xytext=(y-2*i, 102), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax=ax1)
    
    i += 1

pitch1.annotate(text='   Highest xThreat    '.upper(), xytext=(23.5, 120), xy=(60, 40), ha='center', va='center', color='white',
                     bbox=dict(facecolor='darkcyan', edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=36, ax=ax1)

y = 20
i = 0
for p1, cnt in xT_list:
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
        
    pitch1.annotate(text=p1.upper(), 
                          xytext=(y-2*i, 137.5), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax=ax1)
    pitch1.annotate(text=str(cnt), 
                          xytext=(y-2*i, 102), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax=ax1)
    
    i += 1

pass_lines = pitch1.lines(passes_t1.x_start, passes_t1.y_start,
                         passes_t1.x_end, passes_t1.y_end, lw=passes_t1.width,
                         color=color, zorder=1, ax=ax1)


pass_nodes = pitch1.scatter(passes_t1.x_start, passes_t1.y_start, s=passes_t1.marker_size,
                           color=color2, edgecolors='white', linewidth=1, ax=ax1)

for index, row in passes_t1.iterrows():
    pitch1.annotate(row.no, xy=(row.x_start, row.y_start), color='white', va='center',
                   ha='center', size=30, ax=ax1, fontname='Sans Serif')
    
    
pitch3.annotate(text='  Passes Completed  '.upper(), xytext=(98.5, -20), xy=(60, 40), ha='center', 
                     va='center', color='white',
                     bbox=dict(facecolor='mediumorchid', edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=36, ax = ax3)

y = 95
i = 0
for p1, cnt in comp_passes_list2:
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    pitch1.annotate(text=p1.upper(), 
                          xytext=(y-2*i, -2.5), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax = ax3)
    pitch1.annotate(text=cnt, 
                          xytext=(y-2*i, -38.5), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax = ax3)
    
    i += 1
    
pitch3.annotate(text=' Pass Completion %  '.upper(), xytext=(83.5, -20), xy=(60, 40), ha='center', va='center', color='white',
                     bbox=dict(facecolor='mediumorchid', edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=35, ax = ax3)
y = 80
i = 0
for p1, cnt in pass_comp2:
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    pitch1.annotate(text=p1.upper(), 
                          xytext=(y-2*i, -2.5), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax = ax3)
    pitch1.annotate(text=str(cnt)+'%', 
                          xytext=(y-2*i, -38.5), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax = ax3)
    
    i += 1



pitch3.annotate(text='   Prog Passes Made '.upper(), xytext=(68.5, -20), xy=(60, 40), ha='center', va='center', color='white',
                     bbox=dict(facecolor='mediumorchid', edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=35, ax = ax3)
y = 65
i = 0
for p1, cnt in prog_passes_list2:
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    pitch3.annotate(text=p1.upper(), 
                          xytext=(y-2*i, -2.5), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax = ax3)
    pitch3.annotate(text=cnt, 
                          xytext=(y-2*i, -38.5), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax = ax3)
    
    i += 1

pitch3.annotate(text='    Prog Passes Rec   '.upper(), xytext=(53.5, -20), xy=(60, 40), ha='center', va='center', color='white',
                     bbox=dict(facecolor='mediumorchid', edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=35, ax = ax3)
y = 50
i = 0
for p1, cnt in prog_received_list2:
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    pitch3.annotate(text=p1.upper(), 
                          xytext=(y-2*i, -2.5), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax=ax3)
    pitch3.annotate(text=cnt, 
                          xytext=(y-2*i, -38.5), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax=ax3)
    
    i += 1


pitch3.annotate(text=' Pass Combinations  '.upper(), xytext=(38.5, -20), xy=(60, 40), ha='center', va='center', color='white',
                     bbox=dict(facecolor='mediumorchid', edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=35, ax=ax3)
y = 35
i = 0
for p1, p2, cnt in combos_list2:
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
        
    p2 = p2.replace('-', ' ')
    p2 = p2.split(' ')
    if len(p2) > 2:
        name = "".join([x[0].upper() for x in p2])
        p2 = name
    elif len(p2) == 1:
        p2 = p2[0]
    else:
        p2 = p2[1]

    pitch3.annotate(text=p1.upper() + ' --> ' + p2.upper(), 
                          xytext=(y-2*i, -2.5), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax=ax3)
    pitch3.annotate(text=cnt, 
                          xytext=(y-2*i, -38.5), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax=ax3)
    
    i += 1

pitch3.annotate(text='   Highest xThreat    '.upper(), xytext=(23.5, -20), xy=(60, 40), ha='center', va='center', color='white',
                     bbox=dict(facecolor='mediumorchid', edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=35, ax=ax3)

y = 20
i = 0
for p1, cnt in xT_list3:
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
        
    pitch3.annotate(text=p1.upper(), 
                          xytext=(y-2*i, -2.5), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax=ax3)
    pitch3.annotate(text=str(cnt), 
                          xytext=(y-2*i, -38.5), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 28, ax=ax3)
    
    i += 1

pass_lines2 = pitch3.lines(passes_t2.x_start, passes_t2.y_start,
                         passes_t2.x_end, passes_t2.y_end, lw=passes_t2.width,
                         color=color3, zorder=1, ax=ax3)


pass_nodes2 = pitch3.scatter(passes_t2.x_start, passes_t2.y_start, s=passes_t2.marker_size,
                           color=color4, edgecolors='white', linewidth=1, ax=ax3)

for index, row in passes_t2.iterrows():
    pitch3.annotate(row.no, xy=(row.x_start, row.y_start), color='white', va='center',
                   ha='center', size=30, ax=ax3, fontname='Sans Serif')
    
    
sc_team1 = pitch2.scatter(home_shots.X, home_shots.Y, s=home_shots.xG * 5000,
                         ec='#2B2B2B', color='darkcyan', ax=ax2)
sc_team2 = pitch2.scatter(away_shots.X, away_shots.Y, s=away_shots.xG * 5000,
                         ec='#2B2B2B', color='mediumorchid', ax=ax2)

y=213
for metric, home, away in metrics:
    if float(home)>float(away):
        color_box = "darkcyan"
    elif float(home)<float(away):
        color_box = 'mediumorchid'
    else:
        color_box = ''

    pitch2.annotate(text = metric.upper(),xytext=((left2 + TOP_WIDTH/2 )*100 ,y), xy=(60, 40), ha='center', va='center', color='white',
                         fontname = 'Sans Serif', fontsize=50, ax = ax2)
    if color_box == 'darkcyan':
        pitch2.annotate(text=str(home), xytext=(5,y), xy=(60, 40), ha='left', va='center', color='white',
                         bbox=dict(facecolor='darkcyan', edgecolor='none', boxstyle='round,pad=.5'),
                         fontname = 'Sans Serif', fontsize=40, ax = ax2)
        pitch2.annotate(text=str(away), xytext=(95,y), xy=(60, 40), ha='right', va='center', color='white',
                         fontname = 'Sans Serif', fontsize=40, ax = ax2)
    elif color_box == 'mediumorchid':
        pitch2.annotate(text=str(home), xytext=(5,y), xy=(60, 40), ha='left', va='center', color='white',
                         fontname = 'Sans Serif', fontsize=40, ax = ax2)
        pitch2.annotate(text=str(away), xytext=(95,y), xy=(60, 40), ha='right', va='center', color='white',
                        bbox=dict(facecolor='mediumorchid', edgecolor='none', boxstyle='round,pad=.5'),
                         fontname = 'Sans Serif', fontsize=40, ax = ax2)
    else:
        pitch2.annotate(text=str(home), xytext=(5,y), xy=(60, 40), ha='left', va='center', color='white',
                         fontname = 'Sans Serif', fontsize=40, ax = ax2)
        pitch2.annotate(text=str(away), xytext=(95,y), xy=(60, 40), ha='right', va='center', color='white',
                         fontname = 'Sans Serif', fontsize=40, ax = ax2)
        
    
    y-=13.5
    

fig.savefig('dashboard.png', dpi=150)